In [1]:
import torch 
import torchvision 
from torch.utils.data import  DataLoader,datasets,Subset
from torchvision import transforms, datasets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torchvision.models as models
from torch.autograd import Variable
#import torchtext
#from torchtext.data import get_tokenizer

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

# Main stuff

In [3]:
triplets_id = pd.read_csv('task4_be9ai3nsdj/train_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()
test_triplets_id = pd.read_csv('task4_be9ai3nsdj/test_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()

In [4]:
features = np.load('features_resnet18.npy')
features.shape

(10000, 512)

In [132]:
feature_transformed = features[triplets_id]#.reshape(59515, 512*3)
test_feature = features[test_triplets_id]
#feature_transformed = feature_transformed.transpose([0,2,1])

In [133]:
feature_transformed = torch.tensor(feature_transformed)
test_feature = torch.tensor(test_feature)
#features = torch.tensor(features)

In [134]:
print(feature_transformed.shape, test_feature.shape)

torch.Size([59515, 3, 512]) torch.Size([59544, 3, 512])


In [105]:
import torch.nn as nn
import torch.nn.functional as F

in_dim = feature_transformed.shape[2]
out_dim = 1

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(in_dim, 128) 
        self.hidden2 = nn.Linear(128, 64) 
        self.triplet_loss = nn.TripletMarginLoss(margin=0.5, p=2.0, reduction='none')
        
    def forward(self, x):
        x = F.logsigmoid(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.triplet_loss(x[:,0,:], x[:,1,:], x[:,2,:])
        
        return x

net = Net()

In [106]:
forward = net(feature_transformed)
forward

tensor([0.8284, 0.5372, 0.5219,  ..., 0.3840, 0.4039, 0.3612],
       grad_fn=<ClampMinBackward>)

In [110]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=1e-2)

In [111]:
num_epochs = 15

In [130]:
%%time
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(feature_transformed)
    
    criterion = torch.mean(outputs)
    criterion.backward()
    
    optimizer.step()
    
    print(criterion)

tensor(0.1032, grad_fn=<MeanBackward0>)
tensor(0.1049, grad_fn=<MeanBackward0>)
tensor(0.0997, grad_fn=<MeanBackward0>)
tensor(0.0990, grad_fn=<MeanBackward0>)
tensor(0.0982, grad_fn=<MeanBackward0>)
tensor(0.0950, grad_fn=<MeanBackward0>)
tensor(0.0993, grad_fn=<MeanBackward0>)
tensor(0.0994, grad_fn=<MeanBackward0>)
tensor(0.1036, grad_fn=<MeanBackward0>)
tensor(0.1021, grad_fn=<MeanBackward0>)
tensor(0.0952, grad_fn=<MeanBackward0>)
tensor(0.0930, grad_fn=<MeanBackward0>)
tensor(0.0975, grad_fn=<MeanBackward0>)
tensor(0.0965, grad_fn=<MeanBackward0>)
tensor(0.0911, grad_fn=<MeanBackward0>)
CPU times: user 21.8 s, sys: 3.6 s, total: 25.4 s
Wall time: 6.35 s


In [152]:
torch.sum(net(feature_transformed)<0.5)

tensor(55583)

In [153]:
prediction = (net(test_feature)<0.5).numpy()

In [154]:
prediction = np.array(prediction, dtype=int)

In [157]:
np.shape(prediction)

(59544,)

In [158]:
pd.DataFrame(prediction.T).to_csv('prediction.csv', index=False, header=None)